# 2.7 Przewidywanie zakupów za pomocą drzew decyzyjnych
Mamy wyrobione wstępne intuicje odnośnie zbioru danych, który będziemy analizować, więc możemy przejść do ich modelowania. Zanim jednak tego dokonamy, nasz zbiór wymaga pewnego wstępnego przetwarzania, żeby algorytm uczenia drzewa był w stanie wychwycić zawarte w nim regularności.

In [2]:
import pandas as pd
import numpy as np


In [3]:
purchases_df = pd.read_csv("../data/online_shoppers_intention.csv")
purchases_df.sample(6)

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,Month,OperatingSystems,Browser,Region,TrafficType,VisitorType,Weekend,Revenue
4480,0,0.0,0,0.0,8,726.000000,0.075000,0.108333,0.000000,0.6,May,2,2,1,19,Returning_Visitor,False,False
10994,1,12.0,2,39.5,23,488.750000,0.000000,0.015278,0.000000,0.0,Dec,2,2,1,1,Returning_Visitor,True,False
418,0,0.0,0,0.0,1,0.000000,0.200000,0.200000,0.000000,0.0,Mar,1,1,1,1,Returning_Visitor,True,False
7831,0,0.0,0,0.0,54,941.466667,0.000000,0.003774,26.674282,0.0,Aug,4,5,1,1,Returning_Visitor,False,True
11235,1,11.0,0,0.0,59,4165.138690,0.008333,0.017458,0.000000,0.0,Nov,3,2,3,10,Returning_Visitor,False,False
5319,5,53.5,0,0.0,57,2874.966667,0.003279,0.016721,0.000000,0.0,May,2,2,1,4,Returning_Visitor,False,False


In [4]:
purchases_df.dtypes

Administrative               int64
Administrative_Duration    float64
Informational                int64
Informational_Duration     float64
ProductRelated               int64
ProductRelated_Duration    float64
BounceRates                float64
ExitRates                  float64
PageValues                 float64
SpecialDay                 float64
Month                       object
OperatingSystems             int64
Browser                      int64
Region                       int64
TrafficType                  int64
VisitorType                 object
Weekend                       bool
Revenue                       bool
dtype: object

Niestety implementacja drzew decyzyjnych w scikit-learn nie wspiera zmiennych kategorycznych, więc trzeba ominąć ten problem i zakodować je w postaci numerycznej, czyli najprościej: one hot encoding.

In [5]:
purchases_df = purchases_df.astype({
    "OperatingSystems": "category",
    "Browser": "category",
    "Region": "category",
    "TrafficType": "category",
    "Weekend": "int8",
})
purchases_df.dtypes

Administrative                int64
Administrative_Duration     float64
Informational                 int64
Informational_Duration      float64
ProductRelated                int64
ProductRelated_Duration     float64
BounceRates                 float64
ExitRates                   float64
PageValues                  float64
SpecialDay                  float64
Month                        object
OperatingSystems           category
Browser                    category
Region                     category
TrafficType                category
VisitorType                  object
Weekend                        int8
Revenue                        bool
dtype: object

Co ciekawe, drzewa decyzyjne w przeciwieństwie do niektórych modeli, nie wymagają skalowania zmiennych, więc nie będziemy go dokonywać.

In [6]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

In [7]:
categorical_features = purchases_df.select_dtypes(["object", "category"]).columns
categorical_features

Index(['Month', 'OperatingSystems', 'Browser', 'Region', 'TrafficType',
       'VisitorType'],
      dtype='object')

In [8]:
categorical_transformer = Pipeline(steps=[
    ("one_hot_encoding", OneHotEncoder(handle_unknown="ignore")),
])

In [9]:
numerical_features = purchases_df.select_dtypes([int, float]).columns
numerical_features

Index(['Administrative', 'Administrative_Duration', 'Informational',
       'Informational_Duration', 'ProductRelated', 'ProductRelated_Duration',
       'BounceRates', 'ExitRates', 'PageValues', 'SpecialDay'],
      dtype='object')

In [10]:
preprocessor = ColumnTransformer(transformers=[
    ("categorical", categorical_transformer, categorical_features),
    ("numerical", "passthrough", numerical_features),
])

In [11]:
from sklearn.tree import DecisionTreeClassifier

In [12]:
classifier = Pipeline(steps=[
    ("preprocessing", preprocessor),
    ("decision_tree", DecisionTreeClassifier(criterion="gini")),
])

In [13]:
X = purchases_df.drop(columns="Revenue")
y = purchases_df["Revenue"]

Nie będziemy oczywiście naiwnie nauczać na całym naszym zbiorze, żeby później ogłosić wielki sukces, bo mamy kaskadę warunków logicznych, które świetnie rozpoznają przykłady uczące. Zastosujemy walidację krzyżową, która pozwoli nam sprawdzić zdolności generalizacji naszego drzewa.

In [14]:
from sklearn.model_selection import cross_validate

In [15]:
np.random.seed(714)

cross_validate(classifier, X, y, return_train_score=True)

{'fit_time': array([0.31626058, 0.31572938, 0.25109959, 0.25786114, 0.25496459]),
 'score_time': array([0.02011967, 0.00964284, 0.00986505, 0.01079464, 0.00978374]),
 'test_score': array([0.89416058, 0.8945661 , 0.82887267, 0.81549067, 0.79805353]),
 'train_score': array([1., 1., 1., 1., 1.])}

Świetnie, nasz system zwraca skuteczność rzędu 0,8 dla zbioru testowego! Niestety, ale domyślnie wykorzystywaną metryką jest accuracy, a to nie je st najlepszy sposób mierzenia skuteczności dla tak niezbalansowanego problemu. Może rozpoznajemy tylko klasę negatywną?

In [16]:
np.random.seed(529)

cross_validate(classifier, X, y, scoring="f1", return_train_score=True)

{'fit_time': array([0.30672336, 0.30824804, 0.25233006, 0.25558567, 0.2550807 ]),
 'score_time': array([0.01387572, 0.01223469, 0.01353598, 0.0125742 , 0.01244688]),
 'test_score': array([0.63202247, 0.6070922 , 0.50760234, 0.48394495, 0.46300211]),
 'train_score': array([1., 1., 1., 1., 1.])}

Niestety, choć nasz system poradził sobie idealnie z rozpoznaniem zbioru treningowego, na testowym poległ już totalnie. Powodów takiego stanu rzeczy może być bardzo wiele, począwszy od przeuczenia modelu, aż na braku istnienia jakichkolwiek regularności w naszym zbiorze danych skończywszy. Zobaczmy czy zmiana funkcji pomiaru jakości na entropię wniesie jakąkolwiek zmianę.

In [18]:
classifier_entropy = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("decision_tree", DecisionTreeClassifier(criterion="entropy")),
])

In [20]:
np.random.seed(529)

cross_validate(classifier, X, y, scoring="f1", return_train_score=True)

{'fit_time': array([0.31729388, 0.31500173, 0.25740361, 0.25407505, 0.25567484]),
 'score_time': array([0.01430941, 0.01298308, 0.01198006, 0.01225376, 0.01232815]),
 'test_score': array([0.63202247, 0.6070922 , 0.50760234, 0.48394495, 0.46300211]),
 'train_score': array([1., 1., 1., 1., 1.])}

Okazuje się, że w tym wypadku funkcja pomiaru jakości podziału nie ma większego znaczenia - wyniki są identyczne w obu przypadkach. Spróbujmy zacząć od tego, aby stworzyć klasyfikator wykorzystując zaledwie podzbiór cech oryginalnego zbioru, gdyż część posiadanych przez nas cech może wprowadzać niepotrzebny szum.

### Rekursywna eliminacja cech
Dość prostym pomysłem na odnalezienie najlepszego podzbioru cech uczących jest skorzystanie z tzw. rekursywnej eliminacji cech, która odcina kolejne kolumny bazując na ich wagach. Stąd też metoda ta może zostać zaaplikowana tylko do modeli białoskrzynkowych. Domyślnie niezbędne jest określenie jak wiele cech chcielibyśmy wykorzystać, ale można też połączyć cały proces z walidacją krzyżową, aby odnaleźć taki podzbiór, który maksymalizuje wartość metryki, którą liczymy jakość systemu.

In [21]:
from sklearn.feature_selection import RFECV

In [36]:
rfecv = RFECV(DecisionTreeClassifier(criterion="gini"), cv=5, scoring="f1")
classifier_rfecv = Pipeline(steps=[
    ("preprocessing", preprocessor),
    ("decision_tree", rfecv),
])

In [37]:
classifier_rfecv.fit(X, y)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('categorical',
                                                  Pipeline(steps=[('one_hot_encoding',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  Index(['Month', 'OperatingSystems', 'Browser', 'Region', 'TrafficType',
       'VisitorType'],
      dtype='object')),
                                                 ('numerical', 'passthrough',
                                                  Index(['Administrative', 'Administrative_Duration', 'Informational',
       'Informational_Duration', 'ProductRelated', 'ProductRelated_Duration',
       'BounceRates', 'ExitRates', 'PageValues', 'SpecialDay'],
      dtype='object'))])),
                ('decision_tree',
                 RFECV(cv=5, estimator=DecisionTreeClassifier(),
                       scoring='f1'))])

In [43]:
np.max(rfecv.cv_results_["mean_test_score"])

np.float64(0.5484040840630795)

In [44]:
rfecv.n_features_

np.int64(70)

Udało nam się nauczyć model, który być może daje odrobinę lepsze rezultaty niż domyślny, ale nadal nie jest to skok jakościowy. Drzewa decyzyjne są jednak modelem, którego algorytm uczenia przyjmuje pewne parametry, na które warto zwrócić uwagę.

Aby trochę uprościć pracę w następnych notatnikach, zapiszmy zbiór danych do parquet.

In [34]:
purchases_df.to_parquet("../data/purchases_df.parquet")

### Podgląd istotności cech
Zobaczmy jeszcze jak istotne okazały się być poszczególne cechy, dzięki czemu będziemy lepiej rozumieć stworzony proces decyzyjny.

In [48]:
rfecv.estimator_.feature_importances_

array([1.47297497e-03, 5.94298565e-03, 2.66923262e-03, 6.43418131e-04,
       4.23669022e-03, 2.61642689e-03, 2.41812532e-02, 2.69773482e-03,
       1.76722771e-03, 2.20087191e-03, 6.27252911e-03, 5.12492056e-03,
       2.35324026e-03, 2.69471703e-04, 1.47846904e-04, 2.84118391e-03,
       6.34335274e-03, 0.00000000e+00, 4.07221844e-03, 2.16075361e-03,
       3.27803115e-03, 1.00087935e-03, 1.01513055e-03, 0.00000000e+00,
       1.22798905e-03, 0.00000000e+00, 5.68835676e-04, 9.64537603e-05,
       7.72038808e-03, 3.30243085e-03, 7.16770001e-03, 6.38704259e-03,
       2.63228173e-03, 3.84556864e-03, 5.51964681e-03, 3.24739103e-03,
       3.79945887e-03, 5.24530553e-03, 5.04872793e-03, 4.63660484e-03,
       6.53216233e-03, 1.73641636e-03, 3.28421733e-03, 1.11463147e-03,
       2.30812754e-03, 0.00000000e+00, 4.38319076e-03, 1.41884472e-03,
       0.00000000e+00, 1.81544758e-03, 5.49295991e-04, 0.00000000e+00,
       5.61361961e-04, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
      

In [51]:
one_hot_features = classifier_rfecv.named_steps["preprocessing"].named_transformers_["categorical"].named_steps["one_hot_encoding"].get_feature_names_out(categorical_features)
final_features = list(one_hot_features) + list(numerical_features)
results_df = pd.DataFrame({"selected": rfecv.support_}, index=final_features)
results_df.query("selected == True")

,selected
Month_Aug,True
Month_Dec,True
Month_Jul,True
Month_June,True
Month_Mar,True
...,...
ProductRelated_Duration,True
BounceRates,True
ExitRates,True
PageValues,True


In [52]:
selected_features_df = results_df.query("selected == True").assign(importance=rfecv.estimator_.feature_importances_)
selected_features_df.sort_values("importance", ascending=False)

,selected,importance
PageValues,True,0.420056
ProductRelated_Duration,True,0.073980
BounceRates,True,0.073324
ExitRates,True,0.071422
Administrative_Duration,True,0.054066
...,...,...
TrafficType_9,True,0.000000
TrafficType_17,True,0.000000
VisitorType_Other,True,0.000000
TrafficType_19,True,0.000000
